In [1]:
import pandas as pd
import os
import time
import jieba
import numpy as np
np.set_printoptions(precision=2) ## 讓浮點數的輸出不會有那麼多小數點
import matplotlib.pyplot as plt
import operator
## TFIDF 套件
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
jieba.load_userdict("E:/魏老計畫/KeyWordsDB/地區關鍵字/location_dict.txt")
#jieba.load_userdict("E:/魏老計畫/t1.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 0.669 seconds.
Prefix dict has been built succesfully.


In [2]:
def get_sgm_text(file_path):
    text = ""
    with open(file_path, "r" , encoding="utf8")as txt:
        for line in txt.readlines():
            for sgm in jieba.cut(line.replace("北", '北')\
                                 .replace("敦", '敦')\
                                 .replace("化", '化')\
                                 .replace("南", '南')\
                                 .replace("路", '路')\
                                 .replace("二", '二')\
                                 .replace("段", '段').strip().strip()):
                #print(sgm.strip())
                if sgm.strip() != "": 
                    text = text + sgm.strip() + " "
    return text

In [3]:
## 只先針對 1個案件處理：1槽車事故案例

file_path = "E:/魏老計畫/txt/1槽車事故案例/"
txt_list = []
for file_name in os.listdir(file_path):
    sub_path = os.path.join(file_path + file_name)
    for file in os.listdir(sub_path):
        if ".txt" in file:
            txt_list.append(get_sgm_text(os.path.join(sub_path,file)))

In [4]:
## Demo 用
test = "E:/魏老計畫/txt/1槽車事故案例/1-1乙二醇槽車翻覆外洩事故/1-1乙二醇槽車翻覆外洩事故.txt"
text = ""
with open(test, "r" , encoding = "utf8")as txt:
    for line in txt.readlines():
        for sgm in jieba.cut(line.replace("北", '北').replace("敦化南路二段", '敦化南路二段').strip()):
            if sgm.strip() != "":
                text = text + " 【 " + sgm.strip() + " 】 "
            
text;

In [5]:
def get_tfidf_table(txt_list):
    ## 將count.vocabulary_的字典資料做排序
    def sort_dict_by_values(count):
        return  sorted(count.vocabulary_.items(), key = operator.itemgetter(1))
    
    count = CountVectorizer()
    bag = count.fit_transform(txt_list)
    tfidf = TfidfTransformer()
    tfidf_result = tfidf.fit_transform(bag)
    
    ##  產生這次TFIDF分析後的DF資料
    df = pd.DataFrame(tfidf_result.toarray(), columns= [x[0] for x in sort_dict_by_values(count)])
    return df

# 1槽車事故案例  的 TFIDF

In [6]:
new_text = '敬鵬大火》PCB是失火頻率最高的電子業\n桃園敬鵬工廠大火持續燃燒，打火弟兄悲痛中接力救火。（記者李容萍攝）\n首次上稿 13:13更新時間 16:52\n〔記者卓怡君／台北報導〕PCB廠敬鵬的一場惡火，釀成多位消防員殉職的巨禍，根據保險商業同業公會的資料顯示，PCB印刷電路板是最易發生火災的電子業，PCB業人士分析，PCB是相當危險的行業，因PCB在製程中需要使用化學藥劑進行電鍍、蝕刻，工廠內存放大量化學藥劑，只要處理不當或是電線走火就會釀災，同時，因化學溶劑屬危險物品，也很容易發生工安意外。\nPCB工廠火災時有所聞，過去PCB產業曾流傳每年平均要燒掉一座廠，台灣多家PCB廠都曾發生火災，包括敬鵬、華通、健鼎、 金像電、楠梓電、先豐、燿華、耀文、聯茂等，海外PCB廠也是火災頻傳，金像電的常熟廠、志超的蘇州廠、健鼎的無錫廠、南亞的昆山廠、欣興的德國廠，三星的PCB廠也有多次火災，聯茂更曾因為大火，延後股票上市時程。\nPCB業人士指出，PCB工作流程的工作站長達20到30站，製程中的電鍍、蝕刻是最容易引發火災的部份，依過去經驗，起火原因多是化學藥劑處理不當、溫度控制不佳、電線走火，很容易引發不幸事件。\n'
text = ""
for line in new_text.split():
    for sgm in jieba.cut(line.replace("北", '北')\
                         .replace("敦", '敦')\
                         .replace("化", '化')\
                         .replace("南", '南')\
                         .replace("路", '路')\
                         .replace("二", '二')\
                         .replace("段", '段').strip().strip()):
        #print(sgm.strip())
        if sgm.strip() != "": 
            text = text + sgm.strip() + " "

In [7]:
df = get_tfidf_table([text])

In [8]:
##  輸入第i個文件，可以看到該文件詞彙tfidf的排序
df.transpose().sort_values(0,ascending = False)

,0
pcb,0.65
敬鵬,0.15
工廠,0.15
火災,0.15
容易,0.15
大火,0.15
藥劑,0.15
化學,0.15
健鼎,0.10
引發,0.10


In [18]:
def get_one_case_keywords(case_df):
    Q3 = case_df[case_df != 0].describe()[6]  ## Q3 統計值
    return case_df[case_df > Q3].sort_values(ascending= False).index.values

In [19]:
### 單一文件，前75%關鍵詞
df_trans = df.transpose()
case = df_trans[3]
get_one_case_keywords(case)

array(['匝道', 'x1', '路面', '吸油', '木屑', 'x2', '清除', 'x5', '草木', '受污', 'x3',
       '公司', '除油劑', '肇事', '攔油', 'the', '挖土', '貨運', 'of', '情形', '合法',
       '10000psi', '下方', '四號', '表面', '地受', '油罐', '公秉', '事故', '系統', '污染',
       '稻作', '30t', '及表', '20t', '環線', '橋柱', 'x8', '2005', 'on', '25',
       '06', '22', '意外', 'incidents', 'emergency', 'toxic', 'responses',
       '全國', '研討會', 'national', '毒災', '應變案', 'conference', 'chemical',
       'gps', '一處', '經國道', '表土', '農田', '營運', '焚化', '油外', '失控', 'x24',
       '三號', '集中', '東向銜', '中港', '主管', '收集', '土地', '伸縮', '接國道', '平方公尺',
       '割草', '割除', '北向', '土壤', '駕駛員', '所有', '現場', '30', '20', '污水', '16',
       '緊急', '載運', '12', '94', '應變', '排水', '23', '道路', '安全'], dtype=object)

In [20]:
### 全部的文件，前75%關鍵詞

key_words = get_one_case_keywords(df_trans.sum(1))
print("共有：%d 個 Key_words"%len(key_words))
print()
print(key_words)

共有：368 個 Key_words

['事故' 'the' '公司' '現場' 'of' '污染' 'on' 'a3' '毒災' '應變' '匝道' '2005' '翻覆' 'x1'
 'chemical' '路面' 'incidents' 'responses' 'national' 'toxic' 'emergency'
 '全國' '研討會' '應變案' 'conference' '公秉' '情形' '消防' '丙烯醇' '相片' '作業' '吸油' '15'
 '中心' '清除' '柴油' '安全' '液碱' '10' 'a1' '發生' '水道' '二甲苯' '原因' 'and' '攔油' '道路'
 '人員' '外洩' '木屑' '乙二醇' 'to' '液鹼' '屏東縣' '緊急' '司機' '30' '運輸' 'tank' 'a2' '交通'
 '北上' '17' '駕駛員' '台中港' '要求' 'truck' 'ethylene' '救援' 'glycol' '完成' '造成'
 '出口' '通報' '開始' '環保局' '移槽' '起火' '扶正' '工作' '油罐' '照片' '支援' '確認' '20' '吸附'
 '2200' 'x2' '單位' '環境' 'x3' '慶安' '小溪' '外圍' '程序' '南區' '地面' '範圍' '彰化' '十字路口'
 'x5' '草木' '受污' '94' '土壤' '將槽' '肇事' 'eg' '吊起' 'release' '檢查' 'overturn'
 '水溝' '出動' '彰化縣' '25' '50' '波及' '分析' 'was' '派員' '除油劑' '貨運' '系統' '可燃性' '引火'
 '火災' '偵測' '14' '完畢' 'car' '路段' '16' '所有' '路口' '06' '43' '花圃' '吸取' '調派'
 '充作' '甲醇' '除污' '灌充' 'alcohol' '西碼頭' 'allyl' '閥門' '13' '挖土' '協助' '抵達' '00'
 '本次' '持續' '載運' '12' '清洗' '疑似' '涵管' '61' '相關' '一輛' '意外' '灑水' 'no' '圍堵'
 '燃燒' 'diesel' '排空' '爆炸' '擴大' '

# 目前全部的Pdf 進行TFIDF 分析

In [37]:
## 只先針對 1個案件處理：1槽車事故案例

file_path = "E:/魏老計畫/txt/"
txt_list = []
for folder in os.listdir(file_path):
    sub_folder_path = os.path.join(file_path , folder)
    for sub_file in os.listdir(sub_folder_path):
        file_p = os.path.join(sub_folder_path, sub_file)
        for file in os.listdir(file_p):
            if ".txt" in file:
                txt_list.append(get_sgm_text(os.path.join(file_p,file)))

In [39]:
all_df = get_tfidf_table(txt_list)

In [40]:
### 全部的文件，前75%關鍵詞

key_words = get_one_case_keywords(all_df.transpose().sum(1))
print("共有：%d 個 Key_words"%len(key_words))
print()
for idx,kw in enumerate(key_words):
    if idx%10 ==0 :
        print()
    print(kw, end = "\t ")

共有：1239 個 Key_words


事故	 the	 現場	 中心	 of	 公司	 毒災	 on	 2005	 chemical	 
toxic	 消防	 翻覆	 emergency	 鋼瓶	 national	 應變	 incidents	 全國	 研討會	 
conference	 responses	 應變案	 安全	 污染	 發生	 火災	 原因	 and	 fire	 
人員	 變諮詢	 北區	 次氯酸	 in	 造成	 作業	 實驗室	 外洩	 毒災應	 
消防局	 工作	 情形	 氣體	 中區	 ppm	 設備	 波及	 環境	 毒化	 
匝道	 確認	 清除	 儲存	 單位	 tank	 to	 20	 環保局	 氨氣	 
truck	 was	 火勢	 柴油	 a3	 30	 氯氣	 雨水	 10	 危害	 
加藥	 滅火	 扶正	 使用	 13	 nd	 移槽	 工廠	 相關	 氧水	 
回收	 15	 司機	 工業區	 吸油	 完成	 通報	 voc	 爆炸	 路面	 
支援	 pm	 公秉	 導致	 南區	 苯乙烯	 監測	 偵測	 x1	 控制	 
收集	 空氣	 時間	 40	 可燃性	 at	 燃燒	 丙烯醇	 相片	 游泳池	 
起火	 道路	 程序	 94	 污水	 外圍	 交通	 no	 緊急	 液體	 
廢液	 氣閥	 指揮	 範圍	 偵測器	 廢氣	 地面	 50	 噴灑	 倉庫	 
災害	 毒性	 避免	 持續	 運輸	 焚化	 甲苯	 物質	 檢查	 江縣	 
電線	 12	 藥劑	 疑似	 過程	 木屑	 改善	 操作	 accident	 分析	 
協助	 攔油	 環保署	 標準	 overturn	 vcm	 照片	 incident	 駕駛員	 撲滅	 
水霧	 儀器	 管線	 06	 進入	 a1	 化學物質	 液碱	 附近	 液位	 
發現	 引火	 容器	 廢棄	 是否	 from	 防護	 00	 occurred	 廢棄物	 
屏東縣	 aq	 學品	 土壤	 廢水	 二甲苯	 company	 學物質	 開始	 17	 
直接	 產生	 擴大	 檢測	 無法	 移出	 11	 系統	 所有	 界限	 
水道	 乙二醇	 災因	 ph	 自動	 by	 抽除	 had	 充填	 圍堵	 
火警	